<a href="https://colab.research.google.com/github/mlaricobar/WebScrapingCourse/blob/master/DMC2019I_NOT2_Web_Scraping_Linio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Nº2: Web Scraping en una Página de Retail


En esta actividad, intentaremos scrapear los datos de un sitio web de una E-commerce [Linio.com.pe](https://www.linio.com.pe) 

<img src="https://github.com/mikolarico/course-images/blob/master/linio-landing-page.png?raw=true" width="700">

Para realizar Web Scraping, realizaremos los siguientes pasos:

1. Configuración del entorno de desarrollo
2. Investigar la fuente de datos
3. Asegurar que no estamos incumpliendo las directrices éticas del Web Scraping
4. Inspeccionar el sitio web para identificar qué datos queremos extraer
5. Extraer los diferentes campos de datos
6. Diseñar nuestra base de datos
7. Obtener y filtrar los datos
8. Analizar los datos, descubrir insights y crear reportes

Para tener un mejor contexto sobre la actividad, se nos ha encomendado una tarea muy importante que consiste en investigar un poco sobre el sector de retail, específicamente el diferenciar los productos por precios, categorías y otras características. Ante ello:

1. ¿Podemos hacer esto?
2. ¿Tenemos data para hacerlo?
3. ¿Una vez que tengamos la data, que podemos hacer?

## [1] Configuración del Entorno de desarrollo

### [1.1] Descripción de los Packages

En esta actividad, haremos uso de las siguientes librerías para poder realizar Web Scraping:

1. [Requests](http://docs.python-requests.org/en/master/): Esta librería nos permitirá hacer peticiones HTTP. Importante para obtener el contenido de una página web (documentos HTML).
2. [re](https://docs.python.org/3/library/re.html): Módulo para encontrar patrones dentro de una cadena de texto.
3. [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/): Módulo para analizar documentos HTML. Esta biblioteca crea un árbol con todos los elementos del documento y puede ser utilizado para extraer información.
4. [lxml](https://lxml.de/): Módulo para procesar documentos XML y HTML.


### [1.2] Instalación de los Packages

In [0]:
# Package Requests. Por defecto ya viene instalada
!pip install requests

In [0]:
# Package Regex. Por defecto ya viene instalada
!pip install regex

In [0]:
# Package Lxml. Por defecto ya viene instalada
!pip install lxml

In [0]:
# Package BeautifulSoup. Por defecto ya viene instalada
!pip install beautifulsoup4

### [1.3] Validación de la Instalación

Una vez instalada las librerías necesarias, hagamos las pruebas correspondientes para verificar que todo esté correctamente instalado.

In [0]:
# importar la librería requests
import requests

In [0]:
# importar la librería BeautifulSoup
from bs4 import BeautifulSoup

In [0]:
import pandas as pd

## [2] Investigar acerca de la Fuente de Datos

Tal como vimos en la descripción del taller, vamos a usar la información de los productos de la página de [linio.com.pe](https://www.linio.com.pe) como Fuente de Datos. Por lo general, en esta página encontraremos información sobre las características, precios, descuentos, etc sobre productos tales como celulares, TVs, consolas, etc. A continuación, daremos un vistazo sobre la página web.

<img src="https://github.com/mikolarico/course-images/blob/master/linio-pages-1.png?raw=true" width="700">


<img src="https://github.com/mikolarico/course-images/blob/master/linio-pages-2.png?raw=true" width="700">


## [3] Asegurar que no estamos incumpliendo las directrices éticas del Web Scraping

Debemos asegurarnos de que podemos utilizar éticamente este sitio web como nuestra fuente de datos. ¿Por qué? Porque queremos ser buenos ciudadanos del sitio web y no queremos hacer nada que obstaculice o entorpezca los servidores de ese sitio. ¿Cómo hacemos esto? Contestar las siguientes preguntas debería ayudar:

*   ¿Son datos públicamente disponibles? - Todos los datos en [https://linio.com.pe](https://www.linio.com.pe) están totalmente disponibles para el público.
*   ¿Robots.txt se preocupa? Revisar el archivo [linio.com.pe/robots.txt](https://www.linio.com.pe/robots.txt). No hay limitaciones para los directorios y páginas que queremos scrapear. 
*   ¿Cómo planeas usar los datos? - Investigación personal y con fines educativos.

Una vez contestadas estas preguntas, podríamos decir que los objetivos de esta actividad estas alineadas con las pautas éticas de Web Scraping.

Como sugerencia, se recomienda responder a estas preguntas antes de Scrapear cualquier sitio web. Nuestro enfoque no debería ser el de violar o alterar el trabajo de otras personas.

<img src="https://github.com/mikolarico/course-images/blob/master/linio-robots-txt.png?raw=true" width="700">

In [0]:
sitemap = requests.get("http://sitemap.linio.com/pe/sitemap.xml")

In [0]:
type(sitemap.content)

In [0]:
soup = BeautifulSoup(sitemap.content)

In [0]:
sitemap_list = [e.text for e in soup.find_all(name="loc")]
sitemap_list

In [0]:
sitemap_2 = requests.get("http://sitemap.linio.com/pe/sitemap.2.xml")

In [0]:
soup_2 = BeautifulSoup(sitemap_2.content)

In [0]:
url_list = [e.text for e in soup_2.find_all(name="loc")]
lastmod_list = [e.text for e in soup_2.find_all(name="lastmod")]
changefreq_list = [e.text for e in soup_2.find_all(name="changefreq")]
priority_list = [e.text for e in soup_2.find_all(name="priority")]

In [0]:
len(url_list), len(lastmod_list), len(changefreq_list), len(priority_list)

In [0]:
df_url = pd.DataFrame(data={"URL": url_list, "LASTMOD": lastmod_list, "CHANGEFREQ": changefreq_list, "PRIORITY": priority_list})

In [0]:
df_url.head()

In [0]:
url_list[:10]

In [0]:
url_list, lastmod_list, changefreq_list, priority_list = [], [], [], []
for sm in sitemap_list[1:17]:
  print("SITEMAP URL: {0}".format(sm))
  sm_response = requests.get(sm)
  sm_soup = BeautifulSoup(sm_response.content)
  url_list += [e.text for e in sm_soup.find_all(name="loc")]
  lastmod_list += [e.text for e in sm_soup.find_all(name="lastmod")]
  changefreq_list += [e.text for e in sm_soup.find_all(name="changefreq")]
  priority_list += [e.text for e in sm_soup.find_all(name="priority")]

In [0]:
len(url_list), len(lastmod_list), len(changefreq_list), len(priority_list)

In [0]:
df_url = pd.DataFrame(data={"URL": url_list, "LASTMOD": lastmod_list, "CHANGEFREQ": changefreq_list, "PRIORITY": priority_list})

In [0]:
df_url.shape

In [0]:
df_url.head()

In [0]:
df_url.drop_duplicates(subset=["URL"], inplace=True)
df_url.reset_index(drop=True, inplace=True)

In [0]:
df_url.shape

In [0]:
df_url["LASTMOD"].value_counts()

In [0]:
df_url["CHANGEFREQ"].value_counts()

In [0]:
df_url["PRIORITY"].value_counts()

In [0]:
df_url.loc[df_url["URL"].apply(lambda u: "adidas" in u)].head()["URL"].tolist()

In [0]:
url_list[402050]

## [4] Inspeccionar el sitio web para identificar qué datos queremos extraer

## [5] Extraer los diferentes campos de datos

In [0]:
# Definir Url de la página que vamos a Scrapear
url = "https://www.linio.com.pe/p/gran-capacidad-cartera-billetera-larga-para-hombres-marro-n-va4230"

In [0]:
response = requests.get(url)

In [0]:
response_str = response.content.decode("utf-8")

In [0]:
print("Tipo de dato: {0}".format(type(response_str)))
print(response_str)

## [4.1] Introducción al package Regex

Las expresiones regulares se usan para identificar si un patrón existe en una secuencia dada de caracteres (cadena) o no. Ayudan a manipular datos textuales, que a menudo es un requisito previo para los proyectos de ciencia de datos que involucran la extracción de texto.
En Python, las expresiones regulares son soportadas por el package re o regex.

In [0]:
import re

### Caracteres ordinarios:

Los caracteres ordinarios son las expresiones regulares más simples. Se emparejan exactamente y no tienen un significado especial en su sintaxis de expresión regular. Los caracteres comunes se pueden usar para realizar coincidencias exactas simples:

In [0]:
print("\tpython")
print(r"\tpython")

In [0]:
# Un ejemplo del uso del package re
pattern = r"Machine"
sequence = "Machine Learning"
if re.match(pattern, sequence):
    print("Existe coincidencia!")
else:
    print("No existe coincidencia!")

La función match () devuelve un objeto coincidente si el texto coincide con el patrón. De lo contrario, devuelve `None`.

Por otro lado, aquellas cadenas de caracteres que empiezan con la letra `r` son llamados `raw string` o `cadenas en crudo o sin procesar`. La diferencia con las cadenas es que por ejemplo caracteres especiales como el backslash `\` podrán ser reconocidos como unos caracteres más y no como parte de una sequencia de escape.

In [0]:
# \n es un salto de línea
print(r"\n")
print("\n")

# \t es una tabulación
print(r"\t")
print("\t")

### Caracteres comodín o `wild card`:

Los caracteres comodín son caracteres que no coinciden como se ven, sin embargo tienen un significado especial cuando se usan en una expresión regular.
Los caracteres comodín más usados son:
- `.`: Coincide con cualquier caracter, excepto el caracter de nueva línea.
- `\w`: Coincide con cualquier letra, dígito o guión bajo.
- `\W`: Coincide con cualquier caracter que no forme parte de `\w`
- `\s`: Coincide con un solo caracter en blanco como: espacio, nueva línea, tab.
- `\S`: Coincide con cualquier caracter que no forme parte de \s.
- `\t`: Coincide con tab.
- `\n`: Coincide con una nueva línea.
- `\r`: Coincide con un retorno.
- `\d`: Coincide con el dígito decimal 0-9.
- `^`: Coincide con un patrón al comienzo de la cadena.
- `$`: Coincide con un patrón al final de la cadena.
- `[abc]`: Coincide con a, b o c.
- `[a-zA-Z0-9]`: Coincide con cualquier caracter entre (a a z) o (A a Z) o (0 a 9).
- `\A`: Coincide solo al comienzo de la cadena. Funciona en múltiples líneas también.
- `\b`: Coincide solo con el principio o el final de la palabra.
- `\`: Si el caracter que sigue al backslash es un caracter de escape reconocido, entonces se toma el significado especial del término. Por ejemplo, \n se considera como una nueva línea. Sin embargo, si el caracter que sigue a \ no es un caracter de escape reconocido, entonces el \ se trata como cualquier otro caracter.

La función group () devuelve la cadena que coincide con el patrón. Veamos la aplicación de estos caracteres comodín:

In [0]:
print(re.search(r'Mac.i.e', 'Machine Learning').group())
print(re.search(r'Lea\wn\wng', 'MachineLearning').group())
print(re.search(r'M\Wchin\W', 'M@chin€').group())
print(re.search(r'Machine\sLearning', 'Machine Learning').group())
print(re.search(r'Pyth\Sn', 'Python').group())
print(re.search(r'Machine\tLearning', 'Machine	Learning').group())
print(re.search(r'Pyth\dn', 'Pyth0n').group())
print(re.search(r'^Machine', 'Machine Learning con Python').group())
print(re.search(r'Machine Learning$', 'Machine Learning').group())
print(re.search(r'Python: [1-9] módulos', 'Python: 6 módulos').group())
print(re.search(r'Python: [^6] módulos', 'Python: 5 módulos').group())
print(re.search(r'\A[A-Q]ython', 'Cython').group())
print(re.search(r'\b[A-Z]ython', 'Python Machine Learning').group())
print(re.search(r'Machine\\Learning', 'Machine\Learning').group())
print(re.search(r'Machine\sLearning', 'Machine Learning').group())

Aplicación al contenido HTML. A continuación, se muestra un ejemplo para extraer un elemento de imagen.

In [0]:
# Si queremos obtener el siguiente elemento: <img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII=" data-lazy="//i.linio.com/p/5cdc3cbc04f41eb9d7afa9d46914e034-product.jpg" alt="Gran Capacidad Cartera/ Billetera Larga Para Hombres-Marrón" data-slug="//i.linio.com/p/5cdc3cbc04f41eb9d7afa9d46914e034">
print(re.search(r'<img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII=" data-lazy="//i.linio.com/p/5cdc3cbc04f41eb9d7afa9d46914e034-product.jpg" alt="Gran Capacidad Cartera/ Billetera Larga Para Hombres-Marrón" data-slug="//i.linio.com/p/5cdc3cbc04f41eb9d7afa9d46914e034">', response_str).group())

In [0]:
print(re.search(r'<\Smg src="\wat\w:image/png\Wbase64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII=" data-lazy="//i.linio.com/p/5cdc3cbc04f41eb9d7afa9d46914e034-product.jpg" alt="Gran Capacidad Cartera/ Billetera Larga Para Hombres-Marrón" data-slug="//i.linio.com/p/5cdc3cbc04f41eb9d7afa9d46914e034">', response_str).group())

### Repeticiones

Se vuelve bastante tedioso si buscas encontrar patrones largos en una cadena de caracteres. Afortunadamente, el módulo `re` maneja las repeticiones usando los siguientes caracteres especiales:

- `+`: Comprueba uno o más caracteres desde su izquierda.
- `*`: Comprueba cero o más caracteres desde su izquierda.
- `?`: Comprueba exactamente cero o un caracter a su izquierda.
- `{x}`: Repite x veces.
- `{x,}`: Repite como mínimo x veces
- `{x, y}`: Repite como mínimo **x** veces y máximo **y** veces.

In [0]:
re.search(r'\d{3,6}', '0987654321').group()

In [0]:
re.search(r'\d+', '0987654321 3 Machine Learning').group()

In [0]:
re.search(r'\d*', 'aaaaa').group()

In [0]:
re.search(r'\d?', 'aaaaa').group()

In [0]:
re.search(r'<img.+>', response_str).group()

In [0]:
re.search(r'<img src.+>', response_str).group()

In [0]:
img_tag = re.search(r'<img src.+>', response_str).group()

In [0]:
# Obtener la url
re.search(r'src=".+"', img_tag).group()

### Groups and Grouping

Imaginemos que queremos validar direcciones de correo electrónico y verificar el nombre de usuario y host por separado. Para ello podemos hacer uso de la función group, que nos permite coger partes del texto correspondiente.

In [0]:
correo = "Por favor contactar a: machinelearning@fullstack.com"
match = re.search(r'([\w\.-]+)@([\w\.-]+)', correo)
if match:
    print(match.group(0))
    print(match.group(1))
    print(match.group(2))

In [0]:
match = re.search(r'\w+=\W(.+)\W\s\S+=\W(.+)\W\s\S+=\W(.+)\W\s\S+=\W(.+)\W', img_tag)
match

In [0]:
if match:
    print(match.group(0))
    print(match.group(1))
    print("https:" + match.group(2))
    print(match.group(3))
    print(match.group(4))

En general, las principales funciones que la librería `re` nos proporciona son:
- **search(pattern, string, flags=0)**: Con esta función, se escanea la cadena dada buscando la primera posición donde la expresión regular (o patrón) produce una coincidencia. Devuelve un objeto de coincidencia si se encuentra una coincidencia, de lo contrario devuelve None.

In [0]:
pattern = r"Machine"
sequence = "Machine learning"

re.search(pattern, sequence).group()

- **match(pattern, string, flags=0)**: Devuelve el objeto de coincidencia correspondiente si 0 o más caracteres al comienzo de la cadena coinciden con el patrón. De lo contrario, devuelve None.

In [0]:
pattern = r"L"
sequence1 = "Machine Learning"

# No habrá match dado que "L" no está al inicio de 'Machine Learning'
re.match(pattern, sequence1)

<div class="alert alert-info" role="alert">
  <strong>Nota:</strong> La función `match()` busca una coincidencia solo al comienzo de la cadena (por defecto) mientras que la función `search()` busca una coincidencia en cualquier lugar de la cadena.
</div>

- **findall(pattern, string, flags=0)**: Encuentra todas las coincidencias posibles en toda la cadena y las devuelve como una lista de cadenas. Cada cadena devuelta representa una coincidencia.

In [0]:
mensaje = "Por favor contactar a: machinelearning@dmc.com, deeplearning@dmc.com, datascience@dmc.com, bigdata@dmc.com"

#'correos' es una lista que contiene todas las posibles coincidencias
correos = re.findall(r'[\w\.]+@[\w\.]+', mensaje)
for correo in correos: 
    print(correo)

In [0]:
print(img_tag)

In [0]:
# Obtener todos los valores de los atributos del tag de imagen
re.findall(r'\w+-?\w*="([^"]+)"', img_tag)

- **sub(pattern, repl, string, count=0, flags=0)**: Esta es la función sustituta. Devuelve la cadena obtenida reemplazando o sustituyendo las apariciones no superpuestas más a la izquierda del patrón en cadena por la sustitución reemplazada. Si no se encuentra el patrón, la cadena se devuelve sin cambios.

In [0]:
mensaje = "Por favor contactar a: machinelearning@dmc.com"
nuevo_mensaje = re.sub(r'([\w\.]+)@([\w\.]+)', r'deeplearning@dmc.com', mensaje)
print(nuevo_mensaje)

- **compile(pattern, flags=0)**: Compila un patrón de expresión regular en un objeto de expresión regular. Cuando necesite usar una expresión varias veces en un solo programa, usar la función compile () para guardar el objeto de expresión regular resultante para reutilizar es más eficiente. Esto se debe a que las versiones compiladas de los patrones más recientes pasados a compilar () y las funciones de concordancia a nivel de módulo están en caché.

In [0]:
pattern = re.compile(r'([\w\.]+)@([\w\.]+)')
mensaje = "Por favor contactar a: machinelearning@dmc.com"
pattern.search(mensaje).group()

In [0]:
mensajes = ["Contactarse con: machinelearning@dmc.com",
            "Comunicarse a: deeplearning@dmc.com",
            "Para más información con: datascientist@python.pe",
            "reinforcement@learning.pe es mi correo"]
for mensaje in mensajes:
    print(pattern.search(mensaje).group())

In [0]:
# Es lo mismo que: 
re.search(pattern, mensaje).group()

Una vez que hemos entendido algunos métodos básicos y potentes de las expresiones, podemos empezar a extraer cierta información de las Páginas Web.

In [0]:
print(response_str)

In [0]:
# Obtener todos los tags de imágenes
# Nota: [^<>] signfica todos los caracteres excepto los < y >
img_tag_list = re.findall(r'<img src[^<>]+data-lazy="([^"]+)"[^<>]+>', response_str)
img_tag_list = list(set(img_tag_list))
img_tag_list

In [0]:
# Obtener el nombre del producto
product_name = re.findall(r'<span itemprop="name">(.+)</span>', response_str)[-1]
product_name

In [0]:
# Obtener el precio original
product_original_price = re.search(r'<span class="original-price">(.+)</span>', response_str).group()
product_original_price

In [0]:
# Obtener el valor del descuento
product_discount = re.search(r'<span class="discount">(.+)</span>', response_str).group()
product_discount

In [0]:
# Obtener el precio final
product_final_price = re.search(r'<span class="price-main">(.+)</span>', response_str).group()
product_final_price

In [0]:
# Obtener las características del producto
## Paso 1: Obtener el tag o elemento padre de los elementos que contienen las características
product_features_parent = re.search(r'<div class="product-bg-container col-xs-12">[\W\w]+<a data-anchor="product-description">', response_str).group()
print(product_features_parent, "\n")
product_features_list = re.findall(r'<li>([^<]+)</li>', product_features_parent)
product_features_list

In [0]:
# Si nos damos cuenta al inspeccionar la página web, existen ciertos tags que contiene los metadatos de la página
# Son tag del tipo "meta" y con atributo itempro

meta_features = {t[0]: t[1] for t in re.findall(r'<meta itemprop="(.+)" content="(.+)">', response_str)}
meta_features

In [0]:
product_data = {
            "product_original_price": product_original_price,
            "product_discount": product_discount,
            "product_features_list": product_features_list
            }

In [0]:
product_data.update(meta_features)

In [0]:
product_data

Una vez que hemos extraído los datos de un producto en específico, definamos una función que realice esta extracción de tal forma que reciba como entrada la url de la página del producto.

In [0]:
def extract_data(product_url):
  response = requests.get(product_url)
  response_str = response.content.decode("utf-8")
  
  product_original_price = re.search(r'<span class="original-price">(.+)</span>', response_str).group()
  product_discount = re.search(r'<span class="discount">(.+)</span>', response_str).group()
  product_features_parent = re.search(r'<div class="product-bg-container col-xs-12">[\W\w]+<a data-anchor="product-description">', response_str).group()
  product_features_list = re.findall(r'<li>([^<]+)</li>', product_features_parent)
  
  meta_features = {t[0]: t[1] for t in re.findall(r'<meta itemprop="(.+)" content="(.+)">', response_str)}
  
  product_data = {
            "product_original_price": product_original_price,
            "product_discount": product_discount,
            "product_features_list": product_features_list
            }
  product_data.update(meta_features)
  
  return product_data

Probemos con una url distinta, por ejemplo: 
https://www.linio.com.pe/p/televisor-led-uhd-4k-smart-55-samsung-un55nu7090-negro-s081cn

In [0]:
extract_data("https://www.linio.com.pe/p/televisor-led-uhd-4k-smart-55-samsung-un55nu7090-negro-s081cn")

Nos damos cuenta de que la variable product_features_list esta vacía, y es debido a que los elementos del tipo **li** tienen estilos definidos lo que no permite reconocer el patrón que habíamos definido anteriormente. Esta es una de las desventajas de las expresiones regulares, que ante nuevas casuísticas como que el que acabamos de ver debemos redefinir el patrón.

In [0]:
# Importamos el módulo tqdm para simular un progress bar
from tqdm import tqdm

In [0]:
l_product_data = []
for product_url in tqdm(url_list[:20]):
  print(product_url)
  l_product_data.append(extract_data(product_url))

In [0]:
def extract_data(product_url):
  response = requests.get(product_url)
  response_str = response.content.decode("utf-8")
  
  try:
    product_original_price = re.search(r'<span class="original-price">(.+)</span>', response_str).group()
  except AttributeError:
    product_original_price = None
    
  try: 
    product_discount = re.search(r'<span class="discount">(.+)</span>', response_str).group()
  except AttributeError:
    product_discount = None
    
  try:
    product_features_parent = re.search(r'<div class="product-bg-container col-xs-12">[\W\w]+<a data-anchor="product-description">', response_str).group()
    product_features_list = re.findall(r'<li>([^<]+)</li>', product_features_parent)
  except AttributeError:
    product_features_list = None
  
  meta_features = {t[0]: t[1] for t in re.findall(r'<meta itemprop="(.+)" content="(.+)">', response_str)}
  
  product_data = {
            "product_original_price": product_original_price,
            "product_discount": product_discount,
            "product_features_list": product_features_list
            }
  product_data.update(meta_features)
  
  return product_data

In [0]:
l_product_data = []
for product_url in tqdm(url_list[:500]):
  print(product_url)
  l_product_data.append(extract_data(product_url))

In [0]:
df_product_data = pd.DataFrame(l_product_data)

In [0]:
df_product_data.head()

In [0]:
df_product_data.info()

## [4.3] Introducción al package Beautiful Soup


Beautiful Soup es una librería de Python para extraer datos de archivos HTML y XML. Funciona con su analizador favorito para proporcionar formas idiomáticas de navegar, buscar y modificar el árbol de análisis. Normalmente nos ahorra horas o días de trabajo.

In [0]:
url = "https://www.linio.com.pe/p/smartphone-huawei-mate-20-lite-dual-sim-64gb-black-n3zbn7"

In [0]:
# Hagamos una petición a la url usando el método get del módulo requests
page = requests.get(url)

In [0]:
# Pintemos el contenido de la respuesta usando la propiedad content
page.content

Instanciemos el objeto de la clase BeautifulSoup usando como parámetro el contenido de la respuesta obtenida de la petición

In [0]:
soup = BeautifulSoup(markup=page.content, features='html.parser')

In [0]:
# ¿Qué tipo de dato es soup?
type(soup)

Esto nos devolverá un Objeto de la clase BeautifulSoup, que representa el documento como un estructura de datos anidados.

- **prettify(encoding=None)**: Método que nos permite mostrar de una forma mas legible la estructura de datos anidados.

In [0]:
print(soup.prettify())

In [0]:
# Mostremos el título del html
soup.title

- **find(name=None, attrs={}, recursive=True, text=None, **kwargs)**: Método que nos devuelve solo el primer hijo de esta etiqueta que coincida con los criterios dados.

Por ejemplo, vamos a usar este método para obtener el nombre del producto

In [0]:
soup.find(name="span", attrs={"itemprop": "name"})

Lamentablemente nos damos con la sorpresa, de que también existe otro elemento con las mismas condiciones, por lo que a continuación usaremos la propiedad que estas estructuras tienen al ser anidadas.

Es decir, para esto primero encontraremos un elemento que contenga al elemento donde se encuentra el nombre de producto. Procurar que este elemento sea único o tenga una propiedad que permite identificarlo indistintamente.

In [0]:
name_parent_soup = soup.find(name="div", attrs={"class": "product-title col-xs-12"})
name_parent_soup

In [0]:
name_soup = name_parent_soup.find(name="span", attrs={"itemprop": "name"})
name_soup

- **get_text(separator="") | text | getText(separator="")**: Método que permite extraer las cadenas que estan contenidas en el elemento aplicado.



In [0]:
name_soup.get_text()

In [0]:
name_soup.text

In [0]:
name_soup.getText()

Ahora debemos obtener todos los tags de imágenes

- **find_all(name=None, attrs={}, recursive=True, text=None, limit=None, **kwargs)**: Método que nos devuelve una lista de elementos Tag que cumplen el criterio. Puedes especificar el nombre de la Etiqueta o Tag y cualquier atributo que quieres que cumpla la etiqueta.

Por ejemplo, vamos a usar este método para obtener una lista de elementos que contienen a los elementos **img**

In [0]:
image_soup_list = soup.find_all(name="div", attrs={"id": "image-product"})
image_soup_list

- **get(key, default=None)**: Método que nos devuelve el valor de un atributo de una etiqueta, o el valor por defecto en caso de que no tenga el atributo.

In [0]:
image_list = ["https:" + div.find(name="img").get("data-lazy") for div in image_soup_list]
image_list

In [0]:
# Obtener el precio original
price_soup = soup.find(name="span", attrs={"class": "original-price"})
price_soup

In [0]:
product_original_price = price_soup.get_text()

In [0]:
# Obtener el valor del descuento
discount_soup = soup.find(name="span", attrs={"class": "discount"})
discount_soup

In [0]:
product_discount = discount_soup.get_text()
product_discount

In [0]:
# Obtener el precio final
final_price_soup = soup.find(name="span", attrs={"class": "price-main"})
final_price_soup

In [0]:
product_final_price = final_price_soup.get_text()
product_final_price

In [0]:
# Obtener las características del producto
## Paso 1: Obtener el tag o elemento padre de los elementos que contienen las características
product_features_parent_soup = soup.find(name="div", attrs={"class": "product-bg-container col-xs-12"})
## Paso 2: Iterar sobre la lista de tags del tipo li para obtener las características
product_features_list = [li.get_text() for li in product_features_parent_soup.find_all(name="li")]
product_features_list

In [0]:
# Si nos damos cuenta al inspeccionar la página web, existen ciertos tags que contiene los metadatos de la página
# Son tag del tipo "meta" y con atributo itempro

meta_features_parent = soup.find(name="div", attrs={"class": "product-detail row"})
meta_features_parent

In [0]:
review_soup_parent = soup.find(name="div", attrs={"class": "product-review-container row"})
review_soup_parent

In [0]:
general_review = review_soup_parent.find(name="div", attrs={"class": "review-subtitle"}).find(name="p").get_text()
general_review

In [0]:
detailed_review = {"rating_"+p.get_text(separator=";").split(";")[1].strip(): p.get_text(separator=";").split(";")[-2].strip() for p in review_soup_parent.find(name="div", attrs={"class": "chart-container"}).find_all(name="div", attrs={"class": "chart-progress"})}
detailed_review

In [0]:
meta_features_input = meta_features_parent.find_all(name="meta", attrs={"itemprop": re.compile(r".+")}, recursive=False)
meta_features_input

In [0]:
meta_features = {meta.get("itemprop"): meta.get("content") for meta in meta_features_input}
meta_features

In [0]:
product_data = {
            "product_original_price": product_original_price,
            "product_discount": product_discount,
            "product_features_list": product_features_list,
            "image_list": image_list,
            "general_review": general_review,
            "detailed_review": detailed_review
            }

In [0]:
product_data.update(meta_features)

In [0]:
product_data

In [0]:
def extract_data_soup(product_url):
  response = requests.get(product_url)
  soup = BeautifulSoup(markup=response.content, features='html.parser')
  
  # Image List
  image_soup_list = soup.find_all(name="div", attrs={"id": "image-product"})
  image_list = ["https:" + div.find(name="img").get("data-lazy") for div in image_soup_list]
  # Original Price
  product_original_price = soup.find(name="span", attrs={"class": "original-price"}).get_text()
  # Discount
  product_discount = soup.find(name="span", attrs={"class": "discount"}).get_text()
  # Features
  product_features_parent_soup = soup.find(name="div", attrs={"class": "product-bg-container col-xs-12"})
  product_features_list = [li.get_text() for li in product_features_parent_soup.find_all(name="li")]
  # Meta Features
  meta_features_parent = soup.find(name="div", attrs={"class": "product-detail row"})
  meta_features_input = meta_features_parent.find_all(name="meta", attrs={"itemprop": re.compile(r".+")}, recursive=False)
  meta_features = {meta.get("itemprop"): meta.get("content") for meta in meta_features_input}
  # Reviews
  review_soup_parent = soup.find(name="div", attrs={"class": "product-review-container row"})
  general_review = review_soup_parent.find(name="div", attrs={"class": "review-subtitle"}).find(name="p").get_text()
  detailed_review = {"rating_"+p.get_text(separator=";").split(";")[1].strip(): p.get_text(separator=";").split(";")[-2].strip() for p in review_soup_parent.find(name="div", attrs={"class": "chart-container"}).find_all(name="div", attrs={"class": "chart-progress"})}
  
  
  product_data = {
            "product_original_price": product_original_price,
            "product_discount": product_discount,
            "product_features_list": product_features_list,
            "image_list": image_list,
            "general_review": general_review,
            "detailed_review": detailed_review
            }
  product_data.update(meta_features)
  
  return product_data

In [0]:
extract_data_soup("https://www.linio.com.pe/p/smartphone-huawei-mate-20-lite-dual-sim-64gb-black-n3zbn7")

In [0]:
l_product_data = []
for product_url in tqdm(url_list[:20]):
  print(product_url)
  l_product_data.append(extract_data_soup(product_url))

In [0]:
def extract_data_soup(product_url):
  response = requests.get(product_url)
  soup = BeautifulSoup(markup=response.content, features='html.parser')
  
  # Image List
  try:
    image_soup_list = soup.find_all(name="div", attrs={"id": "image-product"})
    image_list = ["https:" + div.find(name="img").get("data-lazy") for div in image_soup_list]
  except AttributeError:
    image_list = []
  # Original Price
  try:
    product_original_price = soup.find(name="span", attrs={"class": "original-price"}).get_text()
  except AttributeError:
    product_original_price = None
  # Discount
  try:
    product_discount = soup.find(name="span", attrs={"class": "discount"}).get_text()
  except AttributeError:
    product_discount = None
  # Features
  try: 
    product_features_parent_soup = soup.find(name="div", attrs={"class": "product-bg-container col-xs-12"})
    product_features_list = [li.get_text() for li in product_features_parent_soup.find_all(name="li")]
  except AttributeError:
    product_features_list = []
    
  # Meta Features
  meta_features_parent = soup.find(name="div", attrs={"class": "product-detail row"})
  meta_features_input = meta_features_parent.find_all(name="meta", attrs={"itemprop": re.compile(r".+")}, recursive=False)
  meta_features = {meta.get("itemprop"): meta.get("content") for meta in meta_features_input}
  
  # Reviews
  try:
    review_soup_parent = soup.find(name="div", attrs={"class": "product-review-container row"})
    general_review = review_soup_parent.find(name="div", attrs={"class": "review-subtitle"}).find(name="p").get_text()
    detailed_review = {"rating_"+p.get_text(separator=";").split(";")[1].strip(): p.get_text(separator=";").split(";")[-2].strip() for p in review_soup_parent.find(name="div", attrs={"class": "chart-container"}).find_all(name="div", attrs={"class": "chart-progress"})}
  except:
    detailed_review = {}
  
  
  product_data = {
            "product_original_price": product_original_price,
            "product_discount": product_discount,
            "product_features_list": product_features_list,
            "image_list": image_list,
            "general_review": general_review,
            "detailed_review": detailed_review
            }
  product_data.update(meta_features)
  
  return product_data

In [0]:
l_product_data = []
for product_url in tqdm(url_list[:200]):
  print(product_url)
  l_product_data.append(extract_data_soup(product_url))

In [0]:
df_product_data = pd.DataFrame(l_product_data)

In [0]:
df_product_data

In [0]:
??soup.find()
help(soup.find())

#### En la siguiente sesión continuaremos con los siguientes pasos

#### OJO:

In [0]:
page = requests.get("https://www.linio.com.pe/api/catalog/similar/RE256FA0FLFX6LAPE")

In [0]:
page.json()

In [0]:
pd.DataFrame(page.json()).T